In [ ]:
!pip install mediapipe

!pip install keras

from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

dir_train = "/content/drive/MyDrive/Solve-a-thon/dataset/images/train"
dir_test = "/content/drive/MyDrive/Solve-a-thon/dataset/images/test"

def crtdf(dir):
  image_path = []
  labels = []
  for label in os.listdir(dir):
    for imgname in os.listdir(os.path.join(dir,label)):
      image_path.append(os.path.join(dir,label,imgname))
      labels.append(label)
    print(label,"completed")
  return image_path,label

train_data = pd.DataFrame()
train_data['image'],train_data['label'] = crtdf(dir_train)
print(train_data)

test_data = pd.DataFrame()
test_data['image'],test_data['label'] = crtdf(dir_test)
print(test_data)

from tqdm.notebook import tqdm

def ext_features(images):
  features = []
  for image in tqdm(images):
    img = load_img(image,color_mode = "grayscale" )
    img = np.array(img)
    features.append(img)
  features = np.array(features)
  features = features.reshape(len(features),48,48,1)
  return features

train_f = ext_features(train_data['image'])
test_f = ext_features(test_data['image'])

x_train = train_f/255.0
x_test = test_f/255.0

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_data['label'])
y_train = le.transform(train_data['label'])
y_train = to_categorical(y_train,num_classes = 7)
le.fit(test_data['label'])
y_test = le.transform(test_data['label'])
y_test = to_categorical(y_test,num_classes = 7)

model = Sequential()

model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu',input_shape = (48,48,1)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size = (3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size = (3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size = (3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(7,activation = 'softmax'))

model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = 'accuracy')

model.fit(x=x_train,y=y_train,batch_size = 128,epochs = 10,validation_data = (x_test,y_test))

model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

from keras.models import model_from_json
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
def ef(image):
    img = load_img(image, grayscale=True)
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

image = '/content/drive/MyDrive/Solve-a-thon/dataset/images/test/angry/157.jpg'
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)